# Description

READ ME

# Boilerplate

## Dependencies

In [0]:
from databricks.feature_engineering import FeatureEngineeringClient
import mlflow
from pyspark.sql import types as T
from pyspark.sql import functions as F

## Parameters

In [0]:
dbutils.widgets.text("catalog_name", "", "00 - Catalog Name")
dbutils.widgets.text("schema_name", "", "01 - Schema Name")

In [0]:
params = dbutils.widgets.getAll()

for key, value in params.items():
  assert value != "", f"Parameter {key} is empty"

locals().update(params)

# Main

## Scoring Spine

In [0]:
df_spine = spark.sql(f"""
  WITH a1 AS (
    SELECT 
      customer_id
      , billing_month
      , total_invoice
      , LAG(total_invoice) OVER (PARTITION BY customer_id ORDER BY billing_month) lm_invoice
    FROM {catalog_name}.{schema_name}.invoices
  ), a2 AS (
    SELECT
      *
      , (total_invoice - lm_invoice) / lm_invoice AS pct_diff
    FROM a1
  )

  SELECT
    customer_id
    , billing_month
    , CASE
        WHEN pct_diff >= 0.1 THEN 'Increase' 
        WHEN pct_diff <= -0.1 THEN 'Decrease'
        ELSE 'Same'
        END invoice_change
  FROM a2
  WHERE 1=1
    AND billing_month > (SELECT min(billing_month) FROM a2)    
  """)

df_scoring_spine = spark.sql(
  """
  SELECT
    *
  FROM {df_spine}
  WHERE billing_month = (SELECT max(billing_month) FROM {df_spine})""",
  df_spine=df_spine)

df_scoring_spine.limit(10).display()

In [0]:
df_scoring_spine.count()

## Model Scoring (Offline)

In [0]:
mlflow.set_registry_uri('databricks-uc')

fe = FeatureEngineeringClient()

model_uri = f"models:/{catalog_name}.{schema_name}.invoice_increase_model@champion"

predictions = fe.score_batch(
    model_uri=model_uri,
    df=df_scoring_spine,
    result_type=T.StringType()
)


In [0]:
predictions.select('customer_id', 'billing_month', 'invoice_change', 'prediction').display()

In [0]:
(
  predictions
  .select('customer_id', 'billing_month', F.col('prediction').alias('invoice_change'))
  .write.mode("overwrite")
  .saveAsTable(f"{catalog_name}.{schema_name}.customer_invoice_change_predictions")
)